In [1]:
import pygame
import numpy as np
import matplotlib.pyplot as plt
import math
from beatmapparser import *
import beatmapparser
from curve import *
from slidercalc import *

pygame 2.1.2 (SDL 2.0.18, Python 3.8.15)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
parser = beatmapparser.BeatmapParser()
parser.parseFile('leaf.osu')
beatmap_d = parser.build_beatmap()

In [58]:
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)

def dist(p1, p2):
    return (p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2

In [59]:
class Circle:
    def __init__(self, center, startTime, radius=20, approachRate=1000, stacked=0):
        self.center = center
        self.radius = radius
        self.startTime = startTime
        self.approachRate = approachRate
        self.appear = startTime - approachRate
        self.disappear = startTime + approachRate // 4
        self.hit = False
#         self.stacked = stacked
#         if stacked:
#             self.center = (center[0] + 4 * stacked, center[1] + 4 * stacked)
    
    def draw(self, screen, time):
        if self.hit: return
        pygame.draw.circle(screen, WHITE, self.center, self.radius)
        pygame.draw.circle(screen, BLACK, self.center, self.radius - 2)
    
    def draw_approach_circle(self, screen, time):
        if self.hit: return
        if time < self.startTime:
            pygame.draw.circle(screen, WHITE, self.center, self.radius + 3 * (self.startTime / (self.startTime - self.appear) - (1 / (self.startTime - self.appear)) * time) * self.radius, width=2)
    
    def score(self, time):
        mouse_pos = pygame.mouse.get_pos()
        if not self.hit and dist(mouse_pos, self.center) <= self.radius ** 2 and any(pygame.key.get_pressed()):
            if time <= self.startTime - self.approachRate // 12:
                score = 0
            elif time <= self.startTime + self.approachRate // 12:
                score = 300
            elif time <= self.startTime + self.approachRate // 4:
                score = 100
            elif time <= self.startTime + self.approachRate:
                score = 50
            else:
                score = 0
            print(score)
            self.hit = True
        

In [60]:
class Slider:
    def __init__(self, startTime, endTime, points, repeatCount, radius=20, approach=1000):
        self.startTime = startTime
        self.endTime = endTime
        self.appear = startTime - approach
        self.disappear = endTime + approach // 4
        self.approach = approach
        self.points = points
        self.numPoints = len(points)
        self.repeatCount = repeatCount
        self.radius = radius
        
        self.initial_center = points[0]
        
        self.trace = []
        self.hit = False
    
    def draw(self, screen, time):
        # draw slider body
        for t in np.arange(0, 1, 0.001):
            px = sum([self.points[i][0] * (1 - t) ** (self.numPoints - i - 1) * t ** (i) * math.comb(self.numPoints - 1, i) for i in range(self.numPoints)])
            py = sum([self.points[i][1] * (1 - t) ** (self.numPoints - i - 1) * t ** (i) * math.comb(self.numPoints - 1, i) for i in range(self.numPoints)])
            pygame.draw.circle(screen, WHITE, (px, py), self.radius)
        for t in np.arange(0, 1, 0.001):
            px = sum([self.points[i][0] * (1 - t) ** (self.numPoints - i - 1) * t ** (i) * math.comb(self.numPoints - 1, i) for i in range(self.numPoints)])
            py = sum([self.points[i][1] * (1 - t) ** (self.numPoints - i - 1) * t ** (i) * math.comb(self.numPoints - 1, i) for i in range(self.numPoints)])
            pygame.draw.circle(screen, BLACK, (px, py), self.radius - 2)
        
        # these keep the circle in the ending position
        if time < self.startTime:
            pygame.draw.circle(screen, WHITE, self.points[0], self.radius, width=2)
        if time > self.endTime:
            end_points = self.points[0] if self.repeatCount % 2 == 0 else self.points[-1]
            pygame.draw.circle(screen, WHITE, end_points, self.radius, width=2)
        
        # draw sliding hit circle
        if self.startTime < time < self.endTime:
            # repeat logic
            one = (self.endTime - self.startTime) / self.repeatCount 
            rep_num = (time - self.startTime) // one
            time_within_rep = (time - self.startTime) % one
            t = time_within_rep / one
            if rep_num % 2 != 0:
                t = 1 - t
            
            # draw circle
            cx = sum([self.points[i][0] * (1 - t) ** (self.numPoints - i - 1) * t ** (i) * math.comb(self.numPoints - 1, i) for i in range(self.numPoints)])
            cy = sum([self.points[i][1] * (1 - t) ** (self.numPoints - i - 1) * t ** (i) * math.comb(self.numPoints - 1, i) for i in range(self.numPoints)])
            pygame.draw.circle(screen, WHITE, (cx, cy), self.radius, width=2)
            
            # for scoring
            mouse_pos = pygame.mouse.get_pos()
            if dist(mouse_pos, (cx, cy)) <= self.radius ** 2 and any(pygame.key.get_pressed()):
                self.trace.append(True)
            else:
                self.trace.append(False)
    
    def draw_approach_circle(self, screen, time):
        # approach circle for initial point to hit
        if time < self.startTime:
            pygame.draw.circle(screen, WHITE, self.initial_center, self.radius + 3 * (self.startTime / (self.startTime - self.appear) - (1 / (self.startTime - self.appear)) * time) * self.radius, width=2)
    
    def score(self, time):
        return
#         if time <= self.startTime - self.approach // 12:
#             self.hit = True
#         elif time >= self.end_time:
            
                            

In [61]:
# class Stack:
#     def __init__(self, hitObjects):
#         self.hitObjects = sorted(hitObjects, key=lambda x: x.startTime, reverse=True)

#     def draw(self, screen, time):
#         if not self.hitObjects: return
#         for obj in self.hitObjects:
#             obj.draw(screen, time)
        
#         for obj in self.hitObjects:
#             obj.draw_approach_circle(screen, time)
    
#     def score(self, time):
        

In [62]:
class Frame:
    def __init__(self, hitObjects):
        self.hitObjects = sorted(hitObjects, key=lambda x: x.startTime, reverse=True)
    
    def draw(self, screen, time):
        if not self.hitObjects: return
        for obj in self.hitObjects:
            obj.draw(screen, time)
            obj.score(time)
        
        for obj in self.hitObjects:
            obj.draw_approach_circle(screen, time)

In [63]:
def FormatParsedBeatmap(beatmap_d: dict):
    hitObjects = []
    for i in range(500):
        if beatmap_d['hitObjects'][i]['object_name'] == 'slider':
            obj = Slider(
                startTime=beatmap_d['hitObjects'][i]['startTime'],
                endTime=beatmap_d['hitObjects'][i]['end_time'],
                points=beatmap_d['hitObjects'][i]['points'],
                repeatCount=beatmap_d['hitObjects'][i]['repeatCount']
            )
        elif beatmap_d['hitObjects'][i]['object_name'] == 'circle':
#             stacked = 0
#             if beatmap_d['hitObjects'][i - 1]['object_name'] == 'circle':
#                 if beatmap_d['hitObjects'][i - 1]['position'] == beatmap_d['hitObjects'][i]['position']:
#                     stacked = hitObjects[-1].stacked + 1
            obj = Circle(
                center=beatmap_d['hitObjects'][i]['position'],
                startTime=beatmap_d['hitObjects'][i]['startTime'],
#                 stacked=stacked
            )
        hitObjects.append(obj)
    return hitObjects


In [64]:
hitObjects = FormatParsedBeatmap(beatmap_d)

In [65]:
import sys, pygame
from pygame.locals import *
pygame.quit()
pygame.init()

size = width, height = 512, 383
clock = pygame.time.Clock()

screen = pygame.display.set_mode(size, DOUBLEBUF, 16)
screen.fill(BLACK)

run = True

while run:
    for event in pygame.event.get():
        if event.type == pygame.QUIT: 
            run = False
            
    screen.fill(BLACK)
    
    objs = []
    for obj in hitObjects:
        if obj.appear < pygame.time.get_ticks() < obj.disappear:
            objs.append(obj)

    frame = Frame(objs)
    frame.draw(screen, pygame.time.get_ticks())
    
    pygame.display.flip()
    pygame.display.update()
    
clock.tick(60)


300
100
0
300
300
0
300
100
0
300
0
0
0
0
0
0
0
300
300
300
300
300
300
300
300
300
300
300


KeyboardInterrupt: 